## Imports and Constants

In [1]:
import os
import numpy as np
import py_midicsv as pm
from midi2img import midi2image
from PIL import Image
from matplotlib import pyplot as plt
import glob
    
MIDI_PATH = "../midi_files"
IMGS_PATH = "../img_files"
DATA_PATH = "../dataset"
DATA_PIXELS = "pixels.npy"
DATA_IMGS = "imgs.pickle"
IMG_WIDTH = 106
IMG_HEIGHT = 106

## Collect midi files

In [5]:
midi_files = glob.glob(os.path.join(MIDI_PATH, "*mid"))
midis = []
for midi in midi_files:
    midis.append(midi)
print(f"There are {len(midis)} midi files")

There are 292 midi files


### Convert MiDi files to images

In [6]:
os.chdir(IMGS_PATH) # sets the target folder for the images
midis_len = len(midis)
print(f"{midis_len} midi files found")
for i, midi in enumerate(midis):
    print(f"{i+1}/{midis_len}", end='\r')
    try:
        midi2image(midi)
    except KeyboardInterrupt as KI:
        raise KI
    except Exception as e:
        print(f"Error creating image from {midi}")
        raise e
print(f"{midis_len} midi files converted")

### Convert images to required size

In [14]:
midi_images = glob.glob(os.path.join(IMGS_PATH, "*.png"))
midi_images_len = len(midi_images)

print(f"{midi_images_len} image files found")

for i, image in enumerate(midi_images):
    print(f"{i+1}/{midi_images_len}", end='\r')
    try:
        img = Image.open(image)
        img = img.resize((IMG_WIDTH, IMG_HEIGHT), Image.ANTIALIAS)
        img.save(image)
    except KeyboardInterrupt as KI:
        raise KI
    except Exception as e:
        print(f"Error processing image {image}")
        raise e
print(f"{midi_images_len} image files processed")

5396 image files found
5396 image files processed


## Constructing Dataset

In [2]:
image_files = glob.glob(os.path.join(IMGS_PATH, "*.png"))
images_len = len(image_files)

print(f"{images_len} image files found. Converting into np array")

def access_imgages(length):
    pixels = [] # to store pixel values
    imgs = []   # to store images
    for i in range(length):
        print(f"{i+1}/{length}", end='\r')
        image = image_files[i]
        try:
            img = Image.open(image)
            img = img.convert('1')
            
            pix = np.array(img.getdata())
            pix = pix.astype('float32')
            pix /= 255.0

            pixels.append(pix.reshape(IMG_HEIGHT, IMG_WIDTH, 1))
            imgs.append(img)
        except KeyboardInterrupt as KI:
            raise KI
        except Exception as e:
            print(f"Error processing {image}")
            raise e

    print(f"{length} image files converted")
    return np.array(pixels), imgs


pixels, imgs = access_imgages(len(image_files))
print(f"\ndataset : {pixels.shape}")
print(f"imgaes : {len(imgs)}")

5396 image files found. Converting into np array
5396 image files converted

dataset : (5396, 106, 106, 1)
imgaes : 5396


### Save dataset into files

In [4]:
def save_dataset(pixels, imgs):
    # save pixels np array
    np.save(os.path.join(DATA_PATH, DATA_PIXELS), pixels)

    # save imgs list
    import pickle
    saveFile = open(os.path.join(DATA_PATH, DATA_IMGS), 'wb')
    pickle.dump(imgs, saveFile)
    saveFile.close()

def load_dataset():
    # load np array into pixels
    pixels = np.load(os.path.join(DATA_PATH, DATA_PIXELS))

    import pickle
    saveFile = open(os.path.join(DATA_PATH, DATA_IMGS), 'rb')
    imgs = pickle.load(saveFile)
    saveFile.close()

    return pixels, imgs

# save dataset
save_dataset(pixels, imgs)


Check that the pixel values are all normalized between 0 and 1

In [5]:
uniqueVals = np.unique(pixels)
if(uniqueVals.shape[0] == 2 and uniqueVals[0] == 0 and uniqueVals[1] == 1):
    print('Pixel values are normalized')
else:
    raise Exception("Pixel values have not been normalized to 0 and 1 in pixels")

Pixel values are normalized


# Creating the GAN

## Imports

In [6]:
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten,BatchNormalization
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from IPython.display import clear_output
import tensorflow as tf

import os
print(os.environ["LD_LIBRARY_PATH"])

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2022-06-13 07:16:10.206842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/
2022-06-13 07:16:10.206894: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


:/lib/
Num GPUs Available:  0


2022-06-13 07:16:12.846291: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 07:16:12.846711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/
2022-06-13 07:16:12.846954: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/
2022-06-13 07:16:12.847006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/
2022-06-13 07:16:12

### Define Discriminator

In [7]:
def define_discriminator(in_shape = (106, 106, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

### Define Generator

In [8]:
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128 * 53 * 53
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((53, 53, 128)))
    model.add(Dense(1024))
    model.add(Conv2DTranspose(1024, (4,4), strides=(2,2), padding='same'))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(Conv2D(1, (7,7) , padding='same',activation = 'sigmoid'))

    return model



### Define GAN

In [9]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model


#### Functions to generate parameters and data for the GAN

In [10]:
# get random points from the dataset for the discriminator
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y
 
# generate random input for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# generate fake input using the generator for the discriminator
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

## Train the GAN

#### Define training

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=51, n_batch=10):
    print(f"Starting training")
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    for i in range(n_epochs):
        print(f"{i}/{n_epochs} epochs")

        for j in range(bat_per_epo):
            print(f"{j}/{bat_per_epo} batch")

            # generate fake and real samples for discriminator
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            
            # train discriminator on fake + real samples
            d_loss, _ = d_model.train_on_batch(X, y)

            # generate noise for generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            
            # train generator on noice
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
    print('\r')

#### Training the GAN

In [12]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)

train(g_model, d_model, gan_model, np.array(pixels), latent_dim)

2022-06-13 07:16:28.091649: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/prithivi/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-06-13 07:16:28.211013: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 143820800 exceeds 10% of free system memory.
2022-06-13 07:16:28.268617: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 143820800 exceeds 10% of free system memory.
2022-06-13 07:16:28.332663: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 143820800 exceeds 10% of free system memory.
2022-06-13 07:16:28.468537: W ten

Starting training
0/51 epochs
0/20 batch


: 

: 